# Paperspace Gradient: TensorFlow 2 Quick Start for Beginners
Last modified: June 3rd 2022

## Purpose and intended audience

This Quick Start tutorial demonstrates TensorFlow 2 usage in a Gradient Notebook. It is aimed at users who are relatively new to TensorFlow, although we do assume that you have basic knowledge of Python code.

For more advanced users who are familiar with TensorFlow and want a quick overview of a more general model being built on Gradient, see the Quick Start for Advanced Users, `quick_start_advanced.ipynb`.

We use [Keras](https://www.tensorflow.org/guide/keras/overview) within TensorFlow to:

- Build a neural network that classifies images
- Train this neural network
- And, finally, evaluate the accuracy of the model

followed by some next steps that you can take to proceed with using Gradient.

The material is based on the original [TensorFlow 2 Beginner Quick Start](https://www.tensorflow.org/tutorials/quickstart/beginner).

See the end of the notebook for the original copyright notice and license.

## Check that you are on a GPU instance

The notebook is designed to run on a Gradient GPU instance (as opposed to a CPU-only instance). The instance type, e.g., A4000, can be seen by clicking on the instance icon on the left-hand navigation bar in the Gradient Notebook interface. It will say if it is CPU or GPU.

![instance_type.png](instance_type.png)

If the instance type is CPU, you can change it by clicking *Stop Instance*, then the instance type displayed to get a drop-down list. Select a GPU instance and start up the Notebook again.

For help with instances, see the Gradient documentation on [instance types](https://docs.paperspace.com/gradient/more/instance-types) or [starting a Gradient Notebook](https://docs.paperspace.com/gradient/explore-train-deploy/notebooks).

## Download and install TensorFlow 2. 
See the [official TensorFlow install guide](https://www.tensorflow.org/install) for details.

In [ ]:
# Requires the latest pip
!pip install --upgrade pip

# Current stable release for CPU and GPU
!pip install tensorflow

In [ ]:
import tensorflow as tf

Provided that you have started this Notebook on a [Gradient GPU instance](https://docs.paperspace.com/gradient/more/instance-types), Gradient will see the GPU and TensorFlow will utilize it. Using a GPU provides large speedups for many machine learning models.

We can list the available GPUs.

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
gpus

By default, TensorFlow allocates all the memory on a GPU to the model being run. This is fine, until, for example, you run a container like this one that has more than one `.ipynb` notebook on it. Then the second notebook's model may fail due to the GPU being out of memory.

We can help with this by setting the memory used to [grow as needed](https://www.tensorflow.org/guide/gpu#limiting_gpu_memory_growth) rather than being allocated all at once at the start.

In [ ]:
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

## Load and prepare the data

The model shows classification of the handwritten digits 0-9 in the [MNIST dataset](http://yann.lecun.com/exdb/mnist/).

The data are loaded, converted from integers to floating-point numbers, and normalized.

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

## Build and train the model

The model is a simple single dense layer neural network with 10 output classes. We build it using `tf.keras.Sequential` by stacking layers.

Here we separate the model from the step to convert its outputs into probabilities.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [ ]:
predictions = model(x_train[:1]).numpy()
predictions

The `tf.nn.softmax` function converts these logits to "probabilities" for each class.

In [ ]:
tf.nn.softmax(predictions).numpy()

(Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output.)

To train the network, we choose an optimizer and loss function for training.

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
it is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.math.log(1/10) ~= 2.3`.

In [ ]:
loss_fn(y_train[:1], predictions).numpy()

The optimized is the commonly used Adam algorithm.

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method trains the model, adjusting the model parameters to minimize the loss.

In [ ]:
model.fit(x_train, y_train, epochs=5)

## Evaluate the model

The `Model.evaluate` method checks the model's performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [ ]:
model.evaluate(x_test, y_test, verbose=2)

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

## Output predictions

If you want your model to return a probability, you can wrap the trained model, and attach the softmax layer to it.

In [ ]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [ ]:
probability_model(x_test[:5])

## Next steps

To proceed with TensorFlow 2 in Gradient, you can:
    
 - Try out the quick_start_advanced.ipynb notebook in this same container if you want to see a simpler example
 - Look at other Gradient material, such as the [tutorials](https://docs.paperspace.com/gradient/get-started/tutorials-list) and [blog](https://blog.paperspace.com)
 - Start writing your own projects, using our [documentation](https://docs.paperspace.com/gradient) when needed
 
If you get stuck or need help, [contact support](https://support.paperspace.com), and we will be happy to assist.

Good luck!

## Original TensorFlow copyright notice and license
##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.